# Universitat Oberta de Catalunya  
## Grado en Ingeniería Informática  
### Trabajo Final de Grado (TFG)

---

## Sistema de recomendaciones basado en técnicas de aprendizaje automático para ampliar la exploración de géneros musicales 
**Autor:** Marc Fernández Pereira  
**Bajo supervisión de:** Dra. María Moreno de Castro  
**Área:** Inteligencia Artificial  
**Semestre:** Otoño 2025  






---

## Índice

3. Fase de modelado

    - 3.1 Introducción
    - 3.2 Selección de las técnicas de modelado
    - 3.3 Generar un diseño de comprobación
    - 3.4 Diseño del pipeline de modelado
    - 3.5 Desarrollo de los modelos de *clustering*

### Imports, funciones y librerías


In [1]:
# Este import centraliza todas las librerías estándar, scikit-learn y funciones personalizadas
from imports import *


# 3. Fase de modelado


## 3.1 Introducción

El objetivo de esta fase es abordar tanto el desarrollo como la evaluación del modelo mediante distintas técnicas de aprendizaje no supervisado. Después de llevar a cabo un análisis exploratorio del conjunto de datos y de preparar el dataset con el que se trabajará, se procede a integrar distintos algoritmos de *clustering* con el fin de identificar patrones de similitud entre canciones basados en sus características musicales. 

Para cada resultado obtenido con cada uno de los modelos se redactará una breve descripción que permitirá al lector comprender los resultados sin necesidad de dominar los fundamentos básicos de las técnicas del aprendizaje profundo (*deep learning*). Al finalizar la tarea de modelado se elaborará un análisis exhaustivo del rendimiento comparado de los modelos, empleando como criterios los índices de Silhouette, Davies–Bouldin y Calinski–Harabasz (véase apartado 3.3).

Antes de comenzar con el desarrollo de esta fase, conviene **recordar que en el cuaderno `01_exploracion_preparacion_datos.ipynb` se llevó a cabo la preparación del conjunto de datos que servirá como base para el proceso de modelado**.

### 3.1.1 Carga del conjunto de datos preparado para el modelado

Se carga el conjunto de datos preparado en la fase de preprocesamiento de datos. Cabe recordar que este conjunto contiene la información original del Spotify Tracks Dataset además de las variables estandarizadas que se han elegido para el desarrollo del modelo no supervisado. 

In [20]:
# Cargamos el conjunto de datos preparado

df = pd.read_csv('spotify_tracks_clean.csv')

# Observamos tipos de datos
dtypes = df.dtypes.value_counts()
display(dtypes)

df.info()

df.head(3)


float64    15
object      6
int64       5
bool        1
Name: count, dtype: int64

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83463 entries, 0 to 83462
Data columns (total 27 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   track_id           83463 non-null  object 
 1   artists            83463 non-null  object 
 2   album_name         83463 non-null  object 
 3   track_name         83463 non-null  object 
 4   popularity         83463 non-null  int64  
 5   duration_ms        83463 non-null  int64  
 6   explicit           83463 non-null  bool   
 7   danceability       83463 non-null  float64
 8   energy             83463 non-null  float64
 9   key                83463 non-null  int64  
 10  loudness           83463 non-null  float64
 11  mode               83463 non-null  int64  
 12  speechiness        83463 non-null  float64
 13  acousticness       83463 non-null  float64
 14  instrumentalness   83463 non-null  float64
 15  liveness           83463 non-null  float64
 16  valence            834

,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,...,tempo,time_signature,track_genre,popularity_bin,energy_norm,acousticness_norm,danceability_norm,valence_norm,tempo_norm,liveness_norm
0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.461,1,...,87.917,4,acoustic,Popular,-0.677495,-0.873374,0.656091,0.947769,-1.142786,0.702678
1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.166,1,...,77.489,4,acoustic,Popular,-1.821142,1.752518,-0.791717,-0.754144,-1.490193,-0.598829
2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.359,0,...,76.332,4,acoustic,Popular,-1.072926,-0.349845,-0.689918,-1.312584,-1.528738,-0.517802


Una vez verificada la estructura del conjunto de datos, se comprobará que las variables estandarizadas mediante z-score mantienen la escala esperada.

### 3.1.2 Descripción general de las variables utilizadas en el modelado

En la fase de preparación se estandarizaron las variables numéricas mediante la técnica z-score, con el objetivo de homogeneizar las escalas de las características que servirán como entrada para los algoritmos de clustering. A continuación, se presentan sus principales estadísticos descriptivos para verificar que mantienen una media próxima a 0 y una desviación estándar cercana a 1.


In [ ]:
# Seleccionamos las columnas que terminan por _norm
norm_variables = [col for col in df.columns if col.endswith('_norm')]

# Mostramos la descripción de las variables
df[norm_variables].describe().T.round(3)


Variables seleccionadas: ['energy_norm', 'acousticness_norm', 'danceability_norm', 'valence_norm', 'tempo_norm', 'liveness_norm']




,count,mean,std,min,25%,50%,75%,max
energy_norm,83463.0,-0.0,1.0,-2.465,-0.697,0.164,0.858,1.412
acousticness_norm,83463.0,-0.0,1.0,-0.968,-0.921,-0.412,0.881,1.965
danceability_norm,83463.0,-0.0,1.0,-3.167,-0.633,0.074,0.735,2.404
valence_norm,83463.0,-0.0,1.0,-1.768,-0.842,-0.051,0.807,2.011
tempo_norm,83463.0,-0.0,1.0,-4.072,-0.751,-0.006,0.597,4.036
liveness_norm,83463.0,-0.0,1.0,-1.110,-0.611,-0.437,0.318,3.954


Tal y como se observa, todas las variables estandarizadas durante la fase de preprocesamiento presentan valores de media próximos a 0 y una desviación típica cercana a 1. Este resultado confirma que el conjunto de datos ha sido correctamente normalizado y se encuentra preparado para ser utilizado en la fase de modelado.

### 3.1.3 Carga del scaler

Llegados a este punto, se procede a cargar la instancia del Scaler con el que se estandarizaron las variables con la finalidad de poder utilizarlo en la fase de modelado en caso de ser necesario.

In [33]:
scaler = joblib.load('scaler_object.pkl')

pd.DataFrame({'media': scaler.mean_,'desviacion': scaler.scale_},index=scaler.feature_names_in_).round(3).head(6)


,media,desviacion
energy,0.636,0.258
acousticness,0.329,0.340
danceability,0.560,0.177
valence,0.466,0.263
tempo,122.220,30.017
liveness,0.219,0.197


La información anterior muestra que se ha cargado el scaler correctamente.

## 3.2 Selección de las técnicas de modelado

A continuación se propone el conjunto de técnicas de aprendizaje no supervisado que se abordarán en este proyecto:

### 3.2.1 K-Means

El algoritmo K-Means es un proceso iterativo que busca particionar el conjunto de datos en $k$ grupos con el objetivo minimizar la suma de distancias cuadráticas entre los puntos y los centroides de cada grupo formado. En cada iteración, las referencias de los centroides se actualizan con la media de los puntos asignados hasta que el proceso converge dando lugar a grupos estabilizados.

Para establecer el valor $k$, habitualmente se utilizan diferentes técnicas como el método del codo (*Elbow method*), el criterio de la silueta (*Silhouette Score*) o el índice de Calinski-Harabasz. En este proyecto se decide aplicar las tres técnicas con el objetivo de visualizar **cómo varía la calidad de agrupamiento** en función del número de clústeres y determinar el valor de $k$ que ofrezca el mejor equilibrio entre los grupos.

El algoritmo K-Means funciona de la siguiente manera:

1. En primer lugar se seleccionan el número de clústeres $k$ utilizando las técnicas anteriores. Que estas técnicas ofrezcan un valor $k$ determinado no implica tener que utilizar únicamente dicho valor, es importante destacar que se pueden generar agrupaciones entorno al valor $k$ que sugieren las técnicas.

2. Acto seguido se inicializan las coordenadas de los centroides. Esto se puede realizar de manera aleatoria o simplemente indicando el valor inicial de manera manual. 

3. Se procede con la asignación de puntos a cada clúster utilizando una métrica de distancia como la euclidiana o la manhattan entre otros.

4. Una vez se tienen los puntos del espacio asignados a un clúster determinado, se actualizan los centroides con la siguiente expresión:  
    $$
    \mathbf{m}_i^{(t+1)} = \frac{1}{|S_i^{(t)}|} \sum_{\mathbf{x}_j \in S_i^{(t)}} \mathbf{x}_j
    $$  
    donde:
    - $\mathbf{m}_i^{(t+1)}$: El nuevo centroide del clúster $i$ en la iteración $t + 1$
    - $|S_i^{(t)}|$: Número de puntos pertenecientes al clúster $i$ en la iteración $t$ 
    - $x_j$: Representa cada punto del clúster 

5. Se repiten de manera iterativa los pasos 3 y 4 hasta que el algoritmo converge.

Para que el algoritmo converja debe ocurrir que los centroides dejen de cambiar de clúster, llegados a este punto se tienen formado los clústeres que k-Means identifica en el conjunto de datos. 


### 3.2.2 K-Medoids

K-Medoids también conocido como Partitioning Around Medoids (PAM) es un algoritmo de agrupamiento que se utiliza para dividir un conjunto de datos en $k$ grupos. A diferencia de K-Means donde se utiliza el cálculo de medias para establecer los centroides en cada iteración, en K-Medoids se utilizan puntos reales del conjunto de datos bajo el nombre de *medoids*. Este algoritmo es realmente eficiente sobre conjuntos de datos que presentan valores atípicos ya que los centroides se eligen como puntos reales del conjunto de datos en lugar de utilizarse las medias.

Para realizar el agrupamiento de los datos a los medoids se utiliza el enfoque de PAM:

1. Al igual que ocurre con k-Means se selecciona un valor $k$ utilizando las técnicas expuestas anteriormente.

2. Se inicializa aleatoriamente los medoides eligiendo $k$ puntos reales del conjunto de datos.

3. Se utiliza una métrica de distancia entre cada dato a los distintos medoides para agruparlo al medoide más cercano.

4. Para cada clúster y una vez se realizan todas las asignaciones, se debe comprobar si alguno de los puntos del reduce el coeficiente de disimilitud, es decir, la suma de distancias dentro del grupo. Si se encuentra 
algún punto que cumpla esta condición se convertirá automáticamente en el nuevo medoide. 

5. Solamente que un medoide cambie, se realizará el proceso de nuevo.

El algoritmo converge cuando todos los medoides no cambian en dos iteraciones seguidas. 


### 3.2.3 DBSCAN y OPTICS

*Density-Based Spatial *Clustering* of Applications with Noise* (DBSCAN) y *Ordering Points To Identify The *Clustering* Structure*(OPTICS) son métodos de *clustering* no supervisado en que la agrupación está basada en la densidad de los datos. El método DBSCAN permite identificar patrones en un juego de datos agrupando los puntos que están más cercanos a alguna métrica como la distancia euclidiana o la manhattan. El método recibe dos parámetros, eps y minPoints:

- **epsilon (eps)** : distancia máxima entre dos puntos para que uno sea considerado vecino del otro

- **minPoints**: Establece el mínimo de puntos para que el método lo considere un clúster

DBSCAN tiene la limitación de que trabaja con agrupación de densidades de datos fijas. El método OPTICS, podría considerarse una extensión de DBSCAN que ofrece la oportunidad de trabajar con agrupación de densidades variables. Además, el parámetro epsilon en OPTICS no determina la formación de clústeres sino que sirve para disminuir la complejidad de cálculo para el algoritmo.


## 3.3 Generar un diseño de comprobación.

Una vez seleccionadas las técnicas de modelado que pueden encajar con el proyecto, es crucial fijar una estrategia de verificación de la calidad del modelo que permita evaluar los resultados obtenidos. La estrategia debe estar enfocada en cómo se evaluarán los resultados con el objetivo no solo de medir el rendimiento, sino también para comprender el modelo a un nivel más profundo. El esquema general de validación estará compuesto por lo siguiente:

### 3.3.1 Validación interna

Una diferencia principal entre los tipos de aprendizaje automático es que el aprendizaje no supervisado no utiliza etiquetas de referencia para generar las agrupaciones. Dado este hecho, existen métricas de rendimiento que permiten evaluar características como la cohesión de las muestras dentro de cada clúster y la separación entre los distintos grupos. A continuación, se proponen tres métricas recomendadas por Montoliu (2021) en Evaluación de modelos no supervisados [31, ver memoria bibliografía del proyecto] para la validación interna de algoritmos no supervisados:

**Coeficiente de Silhouette**
	
El coeficiente intenta describir la similitud entre una muestra y el resto de las muestras de un grupo con los de otro grupo distinto. El resultado de esta métrica se expresa en un rango de [-1, 1], donde 1 indica que la muestra está bien asignada y -1 indica que la muestra está mal asignada. Para obtener una medida global de la calidad del agrupamiento, se calcula el promedio de los coeficientes de Silhouette de todas las muestras del conjunto de datos:

$$
SC = \frac{1}{N} \sum_{i=1}^{N} s(x_i)
$$

donde $N$ es el número total de observaciones y $s(x_i)$ representa el coeficiente de Silhouette individual de cada muestra, definido como:

$$
s(x_i) = \frac{b(x_i) - a(x_i)}{\max(a(x_i), b(x_i))}
$$

donde: 
- $a(x_i)$ es la medida de cohesión, es decvir, la distancia promedio de una muestra a todas las observaciones de su mismo grupo.
 - $b(x_i)$ es la medida de separación, es decir, la distancia mínima promedio de esa muestra a cualquier otro grupo.  

Cuanto mayor sea el valor de $SC$ reflejará alta cohesión interna y buena separación entre grupos lo que se traduce en una mejor será la estructura de los clústeres.


**Índice de Davies-Bouldin (DB)**

Este índice se basa en medir la relación entre la dispersión intra-clúster y la separación entre clústeres. Valores bajos indican que los clústeres son más compactos cuyos centroides están bien separados los unos de los otros. Se puede calcular mediante la siguiente ecuación:

$$
DB = \frac{1}{k} \sum_{i=1}^{k} \max_{j \neq i} \left( \frac{S_i + S_j}{d_{ij}} \right)
$$

donde $d_{ij}$ es la distancia entre los centroides de los grupos $G_i$ y $G_j$,  
$S_i$ es la distancia promedio entre cada punto del grupo $G_i$ y su centroide $\mu_i$,  
y $S_j$ es la distancia promedio entre cada punto del grupo $G_j$ y su centroide $\mu_j$.  

Cuanto más bajo sea el valor del índice $DB$, más compactos y mejor separados estarán los clústeres.


**Índice de Calínski-Haranaz(CH)**

Para obtener este índice se utilizan los índices basados SSW (Sum of squared within) y SSB (Sum of squared between). El SSW se utiliza para medir la cohesión interna de los grupos obtenidos y SSB se emplea para medir la separación entre los grupos obtenidos. Las ecuaciones que definen a estos índices son las siguientes.

$$
SSW = \sum_{i=1}^{k} \sum_{x_j \in G_i} (x_j - \mu_i)^2
$$

donde $k$ es el número de clústeres, $x_j$ una muestra del grupo $G_i$ y $\mu_i$ es el centroide del $i$-ésimo grupo $G_i$.

$$
SSB = \sum_{i=1}^{k} |G_i| (\mu_i - \mu)^2
$$

donde $k$ es el número de clústeres, $|G_i|$ es el número de muestras del grupo $G_i$, $\mu_i$ es el centroide del $i$-ésimo grupo $G_i$ y $\mu$ es la media de todo el conjunto de datos.


Una vez se obtienen los dos índices se podrá obtener el índice CH:


$$
CH = \frac{SSB / (k - 1)}{SSW / (N - k)}
$$

donde $k$ es el número de clústeres y $N$ el número total de observaciones.  
Por lo tanto, cuanto mayor sea el valor de $CH$, mejor reflejará el equilibrio entre una **alta separación entre clústeres** ($SSB$) y una **baja dispersión interna** ($SSW$).


**Estas métricas se calcularán para cada configuración evaluada** de K-Means y K-Medoids, así como sobre los resultados finales de DBSCAN y OPTICS. La comparación conjunta permitirá seleccionar el modelo que ofrezca el mejor equilibrio entre cohesión y separación, en consonancia con el objetivo de generar recomendaciones diversas y coherentes.

### 3.3.2 Validación externa

En este proyecto no se emplea la etiqueta de género musical como variable de entrada en el proceso de *clustering*, ya que el objetivo es descubrir patrones de similitud entre canciones independientemente del género al que pertenecen. Sin embargo, este campo se podría utilizar como etiqueta externa para comparar la similitud entre diferentes métodos de *clustering* mediante el Índice de Rand Ajustado (ARI) [30]. 
El ARI intenta expresar qué proporción de asignaciones del clúster son correctas. Para ello, el algoritmo calcula una medida de similitud entre métodos de *clustering* considerando todos los pares de muestras y los compara contra las verdaderas etiquetas de los clústers. El resultado que arroja ARI está comprendido en un rango de -1 y 1, donde 1 indica coincidencia perfecta, 0 equivale al acuerdo esperado por azar y valores cercanos a -1 reflejan disimilitud. 
Dado que uno de los objetivos del TFG es promover la diversidad y romper la burbuja de género musical, ARI se interpretará de la siguiente manera:
-	Valores altos sugerirán que las agrupaciones reproducen en gran medida las categorías de género tradicionales.
-	Valores bajos indicarán que las agrupaciones descubren similitudes transversales entre canciones de distintos géneros.

El Índice de Rand Ajustado (ARI) se calculará únicamente como contraste adicional: permitirá comprobar si las agrupaciones obtenidas replican los géneros musicales tradicionales o, por el contrario, destapan combinaciones transversales acordes con el objetivo de ampliar la exploración musical.


## 3.5 Desarrollo de los modelos de *clustering*

En este apartado se implementarán los algoritmos seleccionados en la fase de modelado siguiendo el pipeline definido en el apartado anterior. Cada método de agrupamiento contará con su propio subapartado, donde se documentarán los parámetros explorados, las métricas obtenidas y la interpretación de los clústeres resultantes en relación con el objetivo de promover la diversidad musical.

### 3.5.1 K-Means 


<!-- # 3. Fase de modelado


## 3.1 Introducción

El objetivo de esta fase es abordar tanto el desarrollo como la evaluación del modelo mediante distintas técnicas de aprendizaje no supervisado. Después de llevar a cabo un análisis exploratorio del conjunto de datos y de preparar el dataset con el que se trabajará, se procede a aplicar los algoritmos de *clustering* seleccionados con el fin de identificar patrones de similitud entre canciones basados en sus características musicales. Para cada resultado obtenido con cada uno de los modelo se redactará una breve descripción que permitirá al lector comprender los resultados sin necesidad de dominar los fundamentos básicos de las técnicas del aprendizaje profundo (*deep learning*). Al finalizar la tarea de modelado, se elaborará un análisis exhaustivo sobre el rendimiento de cada modelo para comparar cuál ofrece mejor resultado. 

## 3.2 Selección de las técnicas de modelado

A continuación se propone el conjunto de técnicas de aprendizaje no supervisado que se abordarán en este proyecto:

### 3.2.1 K-Means

El algoritmo K-Means es un proceso iterativo que busca particionar el conjunto de datos en $k$ grupos con el objetivo minimizar la suma de distancias cuadráticas entre los puntos y los centroides de cada grupo formado. En cada iteración, las referencias de los centroides se actualizan con la media de los puntos asignados hasta que el proceso converge dando lugar a grupos estabilizados.

Para establecer el valor $k$, habitualmente se utilizan diferentes técnicas como el método del codo (*Elbow method*), el criterio de la silueta (*Silhoutte Score*) o el índice de Calinski-Harabasz. En este proyecto se decide aplicar las tres técnicas con el objetivo de visualizar **cómo varía la calidad de agrupamiento** en función del número de clústeres y determinar el valor de $k$ que ofrezca el mejor equilibrio entre los grupos.

En algoritmo K-Means funciona de la siguiente manera:

1. En primer lugar se seleccionan el número de clústeres $k$ utilizando las técnicas anteriores. Que estas técnicas ofrezcan un valor $k$ determinado no implica tener que utilizar únicamente dicho valor, es importante destacar que se pueden generar agrupaciones entorno al valor $k$ que sugieren las técnicas.

2. Acto seguido se inicializan las coordenadas de los centroides. Esto se puede realizar de manera aleatoria o simplemente indicando el valor inicial de manera manual. 

3. Se procede con la asignación de puntos a cada clúster utilizando una métrica de distancia como la euclidiana o la manhattan entre otros.

4. Una vez se tienen los puntos del espacio asignados a un clúster determinado, se actualizan los centroides con la siguiente expresión:  
    $$
    \mathbf{m}_i^{(t+1)} = \frac{1}{|S_i^{(t)}|} \sum_{\mathbf{x}_j \in S_i^{(t)}} \mathbf{x}_j
    $$  
    donde:
    - $\mathbf{m}_i^{(t+1)}$: El nuevo centroide del clúster $i$ en la iteración $t + 1$
    - $|S_i^{(t)}|$: Número de puntos pertenecientes al clúster $i$ en la iteración $t$ 
    - $x_j$: Representa cada punto del clúster 

5. Se repiten de manera iterativa los pasos 3 y 4 hasta que el algoritmo converge.

Para que el algoritmo converja debe ocurrir que los centroides dejen de cambiar de clúster, llegados a este punto se tienen formado los clústeres que k-Means identifica en el conjunto de datos. 


### 3.2.2 K-Medoids

K-Medoids también conocido como Partitioning Around Medoids (PAM) es un algoritmo de agrupamiento que se utiliza para dividir un conjunto de datos en $k$ grupos. A diferencia de K-Means donde se utiliza el cálculo de medias para establecer los centroides en cada iteración, en K-Medoids se utilizan puntos reales del conjunto de datos bajo el nombre de *medoids*. Este algoritmo es realmente eficiente sobre conjuntos de datos que presentan valores atípicos ya que los centroides se eligen como puntos reales del conjunto de datos en lugar de utilizarse las medias.

Para realizar el agrupamiento de los datos a los medoids se utiliza el enfoque de PAM:

1. Al igual que ocurre con k-Means se selecciona un valor $k$ utilizando las técnicas expuestas anteriormente.

2. Se inicializa aleatoriamente los medoides eligiendo $k$ puntos reales del conjunto de datos.

3. Se utiliza una métrica de distancia entre cada dato a los distintos medoides para agruparlo al medoide más cercano.

4. Para cada clúster y una vez se realizan todas las asignaciones, se debe comprobar si alguno de los puntos del reduce el coeficiente de disimilitud, es decir, la suma de distancias dentro del grupo. Si se encuentra 
algún punto que cumpla esta condición se convertirá automáticamente en el nuevo medoide. 

5. Solamente que un medoide cambie, se realizará el proceso de nuevo.

El algoritmo converge cuando todos los medoides no cambian en dos iteraciones seguidas. 


### 3.2.3 DBSCAN y OPTICS

*Density-Based Spatial Clustering of Applications with Noise* (DBSCAN) y *Ordering Points To Identify The Clustering Structure*(OPTICS) son métodos de clustering no supervisado en que la agrupación está basada en la densidad de los datos. El método DBSCAN permite identificar patrones en un juego de datos agrupando los puntos que están más cercanos a alguna métrica. El método recibe dos parámetros, eps y minPoints:

- epsilon (eps) : distancia máxima entre dos puntos para que uno sea considerado vecino del otro

- minPoints: Establece el mínimo de puntos para que el método lo considere un clúster

DBSCAN tiene la limitación de que trabaja con agrupación de densidades de datos fijas. El método OPTICS, podría considerarse una extensión de DBSCAN que ofrece la oportunidad de trabajar con agrupación de densidades variables. Además, el parámetro epsilon en OPTICS no determina la formación de clústeres sino que sirve para disminuir la complejidad de cálculo para el algoritmo.


 -->
